In [1]:
%run dataFormating.ipynb

C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (18,21,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (14,15,16,17,18,20,21,22,25,26,27,30,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (18,21,22,28,32,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (14,15,16,17,18,20,21,22,25,26,27,30,33,34,35,36,37,

# What subsets of scientific questions tend to be answered correctly by the same subjects?

## Mining

In [2]:
from orangecontrib.associate.fpgrowth import *  
import pandas as pd
from numpy import *

In [3]:
questions = correctedScientific.columns
correctedScientificText = [[] for _ in range(correctedScientific.shape[0])]
for q in questions:
    for r in range(correctedScientific.shape[0]):
        if correctedScientific.loc[r, q]:
            correctedScientificText[r].append(q)
#correctedScientificText

In [4]:
# Get frequent itemsets with support > 25%
# run time < 1 min
itemsets = frequent_itemsets(correctedScientificText, math.floor(len(correctedScientificText) * 0.25))
#dict(itemsets)

In [5]:
# Generate rules according to confidence, confidence > 85 %
# run time < 5 min
rules = association_rules(dict(itemsets), 0.85)
#list(rules)

In [6]:
# Transform rules generator into a Dataframe
rulesDataframe = pd.DataFrame([(ant, cons, supp, conf) for ant, cons, supp, conf in rules])
rulesDataframe.rename(columns = {0:"antecedants", 1:"consequents", 2:"support", 3:"confidence"}, inplace=True)
rulesDataframe.head()

,antecedants,consequents,support,confidence
0,"(Q1, Q15, Q19, Q3, Q7, Q13, Q11)",(Q14),22,1.000000
1,"(Q15, Q19, Q3, Q7, Q13, Q11)","(Q14, Q1)",22,1.000000
2,"(Q15, Q1, Q19, Q3, Q7, Q11)","(Q14, Q13)",22,0.956522
3,"(Q7, Q11, Q15, Q19, Q3)","(Q14, Q13, Q1)",22,0.956522
4,"(Q7, Q1, Q15, Q19, Q3)","(Q14, Q13, Q11)",22,0.956522


In [7]:
# Save the mined rules to file
rulesDataframe.to_csv("results/associationRulesMiningSupport25percentsConfidence85percents.csv")

## Search for interesting rules
Interesting rules are more likely to be the ones with highest confidence, the highest lift or with a bigger consequent set. Pairs can also be especially interesting

In [8]:
# Sort rules by confidence
confidenceSortedRules = rulesDataframe.sort_values(by = ["confidence", "support"], ascending=[False, False])
confidenceSortedRules.head(50)

,antecedants,consequents,support,confidence
5546,(Q15),(Q19),44,1.0
5416,"(Q15, Q1)",(Q19),40,1.0
5298,"(Q14, Q1)",(Q19),38,1.0
5198,"(Q13, Q15)",(Q19),37,1.0
5447,"(Q14, Q15)",(Q19),37,1.0
5259,"(Q11, Q15)",(Q19),36,1.0
4312,"(Q14, Q1, Q15)",(Q19),35,1.0
5320,"(Q14, Q13)",(Q19),35,1.0
4458,"(Q15, Q13, Q1)",(Q19),34,1.0
4506,"(Q14, Q11, Q19)",(Q15),34,1.0


In [9]:
# Sort rules by size of consequent set
rulesDataframe["consequentSize"] = rulesDataframe["consequents"].apply(lambda x: len(x))
consequentSortedRules = rulesDataframe.sort_values(by = ["consequentSize", "confidence", "support"], ascending=[False, False, False])
consequentSortedRules.head(50)

,antecedants,consequents,support,confidence,consequentSize
42,"(Q7, Q3)","(Q14, Q15, Q1, Q19, Q13, Q11)",22,0.916667,6
252,(Q7),"(Q14, Q15, Q1, Q19, Q13, Q11)",24,0.888889,6
88,"(Q7, Q13)","(Q14, Q15, Q1, Q19, Q3, Q11)",22,0.880000,6
90,"(Q7, Q11)","(Q14, Q15, Q1, Q19, Q3, Q13)",22,0.880000,6
913,(Q7),"(Q14, Q15, Q1, Q19, Q3, Q11)",23,0.851852,6
248,"(Q7, Q13)","(Q1, Q14, Q11, Q15, Q19)",24,0.960000,5
250,"(Q7, Q11)","(Q1, Q14, Q13, Q15, Q19)",24,0.960000,5
881,"(Q7, Q3)","(Q1, Q14, Q11, Q15, Q19)",23,0.958333,5
31,"(Q7, Q15, Q3)","(Q1, Q14, Q13, Q11, Q19)",22,0.956522,5
38,"(Q7, Q13, Q3)","(Q1, Q14, Q11, Q15, Q19)",22,0.956522,5


In [10]:
# Select only pairs (rules with antecedent and consequent of size one)
# Sort pairs according to confidence
rulesDataframe["fusedRule"] = rulesDataframe[["antecedants", "consequents"]].apply(lambda x: frozenset().union(*x), axis=1)
rulesDataframe["ruleSize"] = rulesDataframe["fusedRule"].apply(lambda x: len(x))
pairRules = rulesDataframe.sort_values(by=["ruleSize", "confidence", "support"], ascending=[True, False, False])
pairRules.head(30)

,antecedants,consequents,support,confidence,consequentSize,fusedRule,ruleSize
5546,(Q15),(Q19),44,1.000000,1,"(Q15, Q19)",2
5511,(Q7),(Q1),27,1.000000,1,"(Q7, Q1)",2
5513,(Q7),(Q19),27,1.000000,1,"(Q7, Q19)",2
5519,(Q21),(Q26),18,1.000000,1,"(Q21, Q26)",2
5528,(Q20),(Q19),18,1.000000,1,"(Q20, Q19)",2
5539,(Q20),(Q21),18,1.000000,1,"(Q21, Q20)",2
5540,(Q21),(Q20),18,1.000000,1,"(Q21, Q20)",2
5543,(Q21),(Q27),18,1.000000,1,"(Q27, Q21)",2
5545,(Q20),(Q27),18,1.000000,1,"(Q27, Q20)",2
5552,(Q20),(Q26),18,1.000000,1,"(Q20, Q26)",2


In [11]:
# Sort questions by number of apparition in consequents
questions = list(correctedScientific.columns)
for q in questions:
    rulesDataframe[q+"c"] = rulesDataframe["consequents"].apply(lambda x: 1 if q in x else 0)
occurenceInConsequents = rulesDataframe.loc[:,"Q1c":"Q27c"].sum(axis=0)
occurenceInConsequents.sort_values(inplace=True, ascending=False)
occurenceInConsequents

Q19c    1961
Q1c     1893
Q14c    1753
Q15c    1656
Q11c    1628
Q13c    1529
Q3c     1213
Q27c      51
Q26c      51
Q21c      37
Q20c      37
Q4c       11
Q9c        0
Q2c        0
Q5c        0
Q6c        0
Q7c        0
Q8c        0
Q24c       0
Q10c       0
Q23c       0
Q16c       0
Q17c       0
Q18c       0
Q25c       0
Q22c       0
Q12c       0
dtype: int64

In [12]:
# Sort questions by number of apparition in antecedants
for q in questions:
    rulesDataframe[q+"a"] = rulesDataframe["antecedants"].apply(lambda x: 1 if q in x else 0)
occurenceInAntecedants = rulesDataframe.loc[:,"Q1a":"Q27a"].sum(axis=0)
occurenceInAntecedants.sort_values(inplace=True, ascending=False)
occurenceInAntecedants

Q7a     2199
Q3a     1999
Q11a    1979
Q14a    1931
Q15a    1893
Q4a     1879
Q19a    1867
Q13a    1851
Q1a     1820
Q6a      457
Q20a      65
Q21a      65
Q26a      47
Q27a      46
Q23a       0
Q2a        0
Q25a       0
Q24a       0
Q5a        0
Q18a       0
Q8a        0
Q17a       0
Q10a       0
Q22a       0
Q12a       0
Q16a       0
Q9a        0
dtype: int64